# Generate QWI Statistics

## Python Setup

In [ ]:
%pylab inline
# from __future__ import division 
import pandas as pd
import psycopg2
# import seaborn as sns
# sns.set_style("white")
# sns.set_context("poster", font_scale=1.25, rc={"lines.linewidth":1.25, "lines.markersize":8})

## Connection to Database

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
cursor = conn.cursor()

## QWI Functions

In [ ]:
def generate_table(year):
    sql_script = '''
    DROP TABLE IF EXISTS ada_kcmo.qwi_ein_{year}_1;
    CREATE TABLE ada_kcmo.qwi_ein_{year}_1 AS
    select ein
        , count(*) as nb_jobs_current_qtr
        , avg(qwmt) as emp_current_qtr
        , avg(qwmtm4) as emp_4qtrs_ago
        , avg(qwmtm3) as emp_3qtrs_ago
        , avg(qwmtm2) as emp_2qtrs_ago
        , avg(qwmtm1) as emp_prev_qtr
        , avg(qwmtp1) as emp_next_qtr
        , avg(qwbt) as emp_begin_qtr
        , avg(qwet) as emp_end_qtr
        , avg(qwft) as emp_full_qtr
        , avg(qwat) as accessions_current
        , avg(qwa2t) as accessions_consecutive_qtr
        , avg(qwa3t) as accessions_full_qtr
        , avg(qwst) as separations
        , avg(qwht) as new_hires
        , avg(qwrt) as recalls
    from(
        select *
            , case when (qwmtm1 = 1 and qwmt = 1) then 1 else 0 end as qwbt
            , case when (qwmt = 1 and qwmtp1 = 1) then 1 else 0 end as qwet
            , case when (qwmtm1 = 1 and qwmt = 1 and qwmtp1 = 1) then 1 else 0 end as qwft
            , case when (qwmtm1 = 0 and qwmt = 1) then 1 else 0 end as qwat
            , case when (qwmtm1 = 0 and qwmt = 1 and qwmtp1 = 1) then 1 else 0 end as qwa2t
            , case when (qwmtm2 = 0 and qwmtm1 = 1 and qwmt = 1 and qwmtp1 = 1) then 1 else 0 end as qwa3t
            , case when (qwmt = 1 and qwmtp1 = 0) then 1 else 0 end as qwst
            , case when (qwmtm4 = 0 and qwmtm3 = 0 and qwmtm2 = 0 and qwmtm1 = 0) then 1 else 0 end as qwht
            , case when (qwmtm1 = 0 and qwmt = 1 and (qwmtm4=1 or qwmtm3=1 or qwmtm2=1)) then 1 else 0 end as qwrt
        from (
            select *
                , case when wage > 0 then 1 else 0 end as qwmt
                , case when wage_m4 > 0 then 1 else 0 end as qwmtm4
                , case when wage_m3 > 0 then 1 else 0 end as qwmtm3
                , case when wage_m2 > 0 then 1 else 0 end as qwmtm2
                , case when wage_m1 > 0 then 1 else 0 end as qwmtm1
                , case when wage_p1 > 0 then 1 else 0 end as qwmtp1
            from(
                select ein, ssn
                    , count(case when (year = {year} and quarter = 1) then 1 else NULL end) as wage
                    , count(case when (year = {year}-1 and quarter = 1) then 1 else NULL end) as wage_m4
                    , count(case when (year = {year}-1 and quarter = 2) then 1 else NULL end) as wage_m3
                    , count(case when (year = {year}-1 and quarter = 3) then 1 else NULL end) as wage_m2
                    , count(case when (year = {year}-1 and quarter = 4) then 1 else NULL end) as wage_m1
                    , count(case when (year = {year} and quarter = 2) then 1 else NULL end) as wage_p1
                from (
                    select *
                    from kcmo_lehd.mo_wage
                    where year in ({year}-1, {year})
                ) as a
                group by ein, ssn
            ) as b
            where wage > 0
        ) as c
    ) as d
    group by ein
    order by ein;
    
    alter table ada_kcmo.qwi_ein_{year}_1 owner to ada_kcmo_admin;

    commit;
    
    '''.format(year=year)
    
    cursor.execute(sql_script)

## Run Function for Different Years

In [ ]:
generate_table(2014)

In [ ]:
generate_table(2013)

In [ ]:
generate_table(2012)

In [ ]:
generate_table(2011)

In [ ]:
generate_table(2010)

In [ ]:
generate_table(2015)